In [15]:
import numpy as np
import scanpy as sc
import anndata
import scvi
from scib_metrics.benchmark import Benchmarker
%matplotlib inline
import torch

In [16]:
adata = sc.read(
    "data/lung_atlas.h5ad",
    backup_url="https://figshare.com/ndownloader/files/24539942",
)

In [17]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, flavor="cell_ranger", batch_key="batch")
sc.tl.pca(adata, n_comps=30, use_highly_variable=True)

KeyboardInterrupt: 

In [ ]:
adata = adata[:, adata.var.highly_variable].copy()

In [ ]:
adata.obsm["Unintegrated"] = adata.obsm["X_pca"]

In [ ]:
%%capture
import scanorama

# List of adata per batch
batch_cats = adata.obs.batch.cat.categories
adata_list = [adata[adata.obs.batch == b].copy() for b in batch_cats]
scanorama.integrate_scanpy(adata_list)

adata.obsm["Scanorama"] = np.zeros((adata.shape[0], adata_list[0].obsm["X_scanorama"].shape[1]))
for i, b in enumerate(batch_cats):
    adata.obsm["Scanorama"][adata.obs.batch == b] = adata_list[i].obsm["X_scanorama"]

In [ ]:
import pyliger

bdata = adata.copy()
# Pyliger normalizes by library size with a size factor of 1
# So here we give it the count data
bdata.X = bdata.layers["counts"]
# List of adata per batch
adata_list = [bdata[bdata.obs.batch == b].copy() for b in batch_cats]
for i, ad in enumerate(adata_list):
    ad.uns["sample_name"] = batch_cats[i]
    # Hack to make sure each method uses the same genes
    ad.uns["var_gene_idx"] = np.arange(bdata.n_vars)


liger_data = pyliger.create_liger(adata_list, remove_missing=False, make_sparse=False)
# Hack to make sure each method uses the same genes
liger_data.var_genes = bdata.var_names
pyliger.normalize(liger_data)
pyliger.scale_not_center(liger_data)
pyliger.optimize_ALS(liger_data, k=30)
pyliger.quantile_norm(liger_data)


adata.obsm["LIGER"] = np.zeros((adata.shape[0], liger_data.adata_list[0].obsm["H_norm"].shape[1]))
for i, b in enumerate(batch_cats):
    adata.obsm["LIGER"][adata.obs.batch == b] = liger_data.adata_list[i].obsm["H_norm"]

/local/data/anaconda3/envs/BT/lib/python3.11/site-packages/pyliger/preprocessing/_scale.py:88: RuntimeWarning: divide by zero encountered in divide
100%|██████████| 30/30 [10:19<00:00, 20.65s/it]


In [ ]:
adata.write_h5ad(filename="data/adataw.h5ad")

In [18]:
adata = anndata.read_h5ad(filename="data/adataw.h5ad")

In [19]:
def trainModel(adata, prior, prior_kwargs = None, max_epochs = 100):
    scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key="batch")
    vae = scvi.model.SCVI(adata, prior_distribution = prior,prior_kwargs=prior_kwargs, n_layers=2, n_latent=30)
    vae.train(max_epochs=max_epochs,check_val_every_n_epoch=5)
    adata.obsm["scVI"] = vae.get_latent_representation()
    return adata, vae

def plotBenchmarkResults(adata,keys=None):
    if keys == None:
        bm = Benchmarker(
        adata,
        batch_key="batch",
        label_key="cell_type",
        embedding_obsm_keys=["Unintegrated", "LIGER", "Scanorama", "scVI"],
        n_jobs=6,
        )
    else:
        bm = Benchmarker(
        adata,
        batch_key="batch",
        label_key="cell_type",
        embedding_obsm_keys=keys,
        n_jobs=6,
        )
    bm.benchmark()
    bm.plot_results_table(min_max_scale=False)

In [20]:
sdnormalAdata, vaeSD = trainModel(anndata.read_h5ad(filename="data/adataw.h5ad"), "sdnormal",max_epochs=100)

/local/data/scvi-tools/scvi/data/fields/_base_field.py:64: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/100:   0%|          | 0/100 [00:00<?, ?it/s]

/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)


Epoch 2/100:   1%|          | 1/100 [00:01<02:48,  1.70s/it, v_num=1, train_loss_step=614, train_loss_epoch=662]

/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)


Epoch 6/100:   5%|▌         | 5/100 [00:08<02:50,  1.79s/it, v_num=1, train_loss_step=553, train_loss_epoch=562]

/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)


Epoch 49/100:  48%|████▊     | 48/100 [01:22<01:28,  1.69s/it, v_num=1, train_loss_step=560, train_loss_epoch=514]

/local/data/anaconda3/envs/BT/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
normalAdata, vaeN = trainModel(anndata.read_h5ad(filename="data/adataw.h5ad"), "normal",max_epochs=200)

/local/data/scvi-tools/scvi/data/fields/_base_field.py:64: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/200:   0%|          | 0/200 [00:00<?, ?it/s]

/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)


Epoch 2/200:   0%|          | 1/200 [00:01<05:43,  1.72s/it, v_num=1, train_loss_step=571, train_loss_epoch=660]

/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)


Epoch 200/200: 100%|██████████| 200/200 [05:26<00:00,  1.64s/it, v_num=1, train_loss_step=529, train_loss_epoch=517]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [05:26<00:00,  1.63s/it, v_num=1, train_loss_step=529, train_loss_epoch=517]


In [ ]:
mogAdata, vaeMG = trainModel(anndata.read_h5ad(filename="data/adataw.h5ad"), "mixofgaus",max_epochs=200)

/local/data/scvi-tools/scvi/data/fields/_base_field.py:64: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/200:   0%|          | 0/200 [00:00<?, ?it/s]

/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)


Epoch 2/200:   0%|          | 1/200 [00:01<06:26,  1.94s/it, v_num=1, train_loss_step=558, train_loss_epoch=662]

/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)


Epoch 200/200: 100%|██████████| 200/200 [06:21<00:00,  1.92s/it, v_num=1, train_loss_step=539, train_loss_epoch=517]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [06:21<00:00,  1.91s/it, v_num=1, train_loss_step=539, train_loss_epoch=517]


In [ ]:
vampAdata, vaeVP = trainModel(anndata.read_h5ad(filename="data/adataw.h5ad"), "vamp",max_epochs=100)

/local/data/scvi-tools/scvi/data/fields/_base_field.py:64: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/100:   0%|          | 0/100 [00:00<?, ?it/s]

/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)


Epoch 2/100:   1%|          | 1/100 [00:02<03:45,  2.28s/it, v_num=1, train_loss_step=556, train_loss_epoch=660]

/local/data/scvi-tools/scvi/module/_vae.py:519: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)


Epoch 100/100: 100%|██████████| 100/100 [03:35<00:00,  2.16s/it, v_num=1, train_loss_step=483, train_loss_epoch=513]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|██████████| 100/100 [03:35<00:00,  2.15s/it, v_num=1, train_loss_step=483, train_loss_epoch=513]


In [ ]:
def plotBenchmarkResultsAll():
    adataAll = normalAdata
    adataAll.obsm["scVISDNormal"] = sdnormalAdata.obsm["scVI"]
    adataAll.obsm["scVINormal"] = normalAdata.obsm["scVI"]
    adataAll.obsm["scVIMoG"] = mogAdata.obsm["scVI"]
    adataAll.obsm["scVIVamp"] = vampAdata.obsm["scVI"]
    plotBenchmarkResults(adataAll,["Unintegrated", "LIGER", "Scanorama", "scVINormal","scVIMoG","scVIVamp","scVISDNormal"])

In [ ]:
def plotTrainingHistory(model):
    train_elbo = model.history["elbo_train"][1:]
    test_elbo = model.history["elbo_validation"]
    ax = train_elbo.plot()
    test_elbo.plot(ax=ax)

def plotReconstructionLoss(model):
    train_elbo = model.history["reconstruction_loss_train"][1:]
    test_elbo = model.history["reconstruction_loss_validation"]
    ax = train_elbo.plot()
    test_elbo.plot(ax=ax)

def plotKLLocalLoss(model):
    train_elbo = model.history["kl_local_train"][1:]
    test_elbo = model.history["kl_local_validation"]
    ax = train_elbo.plot()
    test_elbo.plot(ax=ax)

def plotKLGlobalLoss(model):
    train_elbo = model.history["kl_global_train"][1:]
    test_elbo = model.history["kl_global_validation"]
    ax = train_elbo.plot()
    test_elbo.plot(ax=ax)